In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#Import required Pythin modules
import time
#start time count
tic = time.perf_counter()
import pandas as pd
from csv import writer
import csv
import re
import sys
import os

#set recursionlimit
sys.setrecursionlimit(10**9) 

# special characters in names
regex = re.compile('[._-]') 

#define variable names of csv and xlsx
dict_excel_file = 'firstnames and lastnames.xlsx'
input_DATABASE = 'DATABASE.csv'

#copy previous OUTPUT.csv to the backup folder
os.system('cmd /c "move OUTPUT.csv BACKUP"')

# import the names "firstnames" file into a data frame
df1 = pd.read_excel(dict_excel_file, sheet_name='Firstnames')

#import the input csv file into a data frame
df2 = pd.read_csv(input_DATABASE)

# import the names "lastnames" file into a data frame
df3 = pd.read_excel(dict_excel_file, sheet_name='Lastnames')

# Keep the DataFrame with valid entries in the same variable for all three dataframes.
df1.dropna(inplace=True)
df2.dropna(inplace=True)
df3.dropna(inplace=True)

#convert the thre data frames into dictionaries
dictionaryDF1 = df1.to_dict()
dictionaryDF2 = df2.to_dict()
dictionaryDF3 = df3.to_dict()

#define variables

#tracker for single and double names

#single names in first names
setSingle1 = set()
#single names in last names
setSingle2 = set()

#double names in first names
dictDouble0 = {}
#double names in last names
dictDouble1 = {}

#printing function1 to OUTPUT.csv file [without formatting]

def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

#print the headers to the OUTPUT.csv file
OutputFileHeaders = ["INPUT","OUTPUT1","OUTPUT2"]
append_list_as_row('OUTPUT.csv', OutputFileHeaders) 

#printing function2 to OUTPUT.csv file [with formatting]

#string1 input
#string2 dictionary
#string3 outcome message

def printToCsv(string1,string2,string3):
     
    LEN1 = len(string1)
    LEN2 = len(string2)
    LEN3 = LEN2/LEN1
    row_contents = str(string1+","+ string2 + " " + string1 + "," +'{0:1.0%}'.format(LEN3)+ "," +string3)
    row_contentsList = list(row_contents.split(",")) 
    print(list(row_contentsList))
    append_list_as_row('OUTPUT.csv', row_contentsList)

#function to check for double names

def dicDoubleFunction(nameStr,dictionaryDF1, dictionaryDF3):
    
    SplitList1 = re.split(regex,nameStr,1)
    
    for key in dictionaryDF1["FIRSTNAME"]: 
        if  dictionaryDF1["FIRSTNAME"][key] == SplitList1[0]:
            dictDouble0[nameStr] = SplitList1[0]
            break
            
    for key in dictionaryDF3["LASTNAME"]: 
        if  dictionaryDF3["LASTNAME"][key] == SplitList1[1]:
            dictDouble1[nameStr] = SplitList1[1]
            break
        
    if (nameStr in dictDouble0 and nameStr in dictDouble1):
        printToCsv(nameStr, SplitList1[0] + " " + SplitList1[1],"Exactly match both names")
        
    elif nameStr in dictDouble0:
        printToCsv(nameStr,SplitList1[0],"Matched firstname only")  
        
    elif nameStr in dictDouble1:
        printToCsv(nameStr,SplitList1[1],"Matched namename only")
        
    elif (nameStr not in dictDouble0 and nameStr not in dictDouble1):
        printToCsv(nameStr,"","Both not found!")
        
#function to check for single names in DF1

def dicSingleFunctionFirst(nameStr,dictionaryDF1):
        
    for key in dictionaryDF1["FIRSTNAME"]:
                        
        if nameStr == dictionaryDF1["FIRSTNAME"][key]:
            printToCsv(nameStr,dictionaryDF1["FIRSTNAME"][key],"Exact match from firstname")
            setSingle1.add(nameStr)
            break
            
        elif nameStr[:-1] == dictionaryDF1["FIRSTNAME"][key]:
            printToCsv(nameStr,dictionaryDF1["FIRSTNAME"][key],"Leading match from firstname")
            setSingle1.add(nameStr)
            break
            
        elif nameStr[1:] == dictionaryDF1["FIRSTNAME"][key]:
            printToCsv(nameStr,dictionaryDF1["FIRSTNAME"][key],"Trailing match from firstname")
            setSingle1.add(nameStr)
            break
            
#function to check for single names in DF3
            
def dicSingleFunctionLast(nameStr,dictionaryDF3):
    
    for key in dictionaryDF3["LASTNAME"]:
        if nameStr == dictionaryDF3["LASTNAME"][key]:
            printToCsv(nameStr,dictionaryDF3["LASTNAME"][key],"Exact match from lastname")
            setSingle2.add(nameStr)
            break
        
        elif nameStr[:-1] == dictionaryDF3["LASTNAME"][key]:
            printToCsv(nameStr,dictionaryDF3["LASTNAME"][key],"Leading match from lastname")
            setSingle2.add(nameStr)
            break
            
        elif nameStr[1:] == dictionaryDF3["LASTNAME"][key]:
            printToCsv(nameStr,dictionaryDF3["LASTNAME"][key],"Trailing match from lastname")
            setSingle2.add(nameStr)
            break

#main look through input file which calls above functions

for key in dictionaryDF2["NAME"]:
      
    if(regex.search(dictionaryDF2["NAME"][key]) == None): 
        dicSingleFunctionFirst(dictionaryDF2["NAME"][key],dictionaryDF1)
        
        if dictionaryDF2["NAME"][key] not in setSingle1:
            dicSingleFunctionLast(dictionaryDF2["NAME"][key],dictionaryDF3)
            if dictionaryDF2["NAME"][key] not in setSingle2:
                printToCsv(dictionaryDF2["NAME"][key],"","Not Found!")
        
    else:
        dicDoubleFunction(dictionaryDF2["NAME"][key],dictionaryDF1,dictionaryDF3)

# timer code

toc = time.perf_counter()

print("I----------------- STATS---------------------------------------------------------I")
print(f"Downloaded the file in {toc - tic:0.2f} seconds")
print("I----------------- STATS---------------------------------------------------------I")  



['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['NEAL', 'NEAL NEAL', '100%', 'Exact match from firstname']
['CRAIG', 'CRAIG CRAIG', '100%', 'Exact match from firstname']
['NATHAN', 'NATHAN NATHAN', '100%', 'Exact match from firstname']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['GIOVANNI', 'GIOVANNI GIOVANNI', '100%', 'Exact match from firstname']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['CARRIE', 'CARRI CARRIE', '83%', 'Leading match from firstname']
['BARRY', 'BARRY BARRY', '100%', 'Exact match from firstname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['EDITOR', ' EDITOR', '0%', 'Not Found!']
['RECRUITER', ' RECRUITER', '0%', 'Not Found!']
['CLIPPER.SUPPORT', ' CLIPPER.SUPPORT', '0%', 'Both not found!']
['TROY.JONAS', 'TROY JONAS TROY.JONAS', '100%', 'Exactly match both names']
['LEE.KRANTZ', 'LEE KRANTZ LEE.KRANTZ', '100%', 'Exactly match both nam

['SKORTAS', ' SKORTAS', '0%', 'Not Found!']
['KEITH.ARFLACK', 'KEITH KEITH.ARFLACK', '38%', 'Matched firstname only']
['BLANKHEET', ' BLANKHEET', '0%', 'Not Found!']
['NPPUTTAQUNTA', ' NPPUTTAQUNTA', '0%', 'Not Found!']
['KHASHIMOTO', ' KHASHIMOTO', '0%', 'Not Found!']
['MJBALOW', ' MJBALOW', '0%', 'Not Found!']
['JRHENGESBACH', ' JRHENGESBACH', '0%', 'Not Found!']
['PMPALAZZOLO', ' PMPALAZZOLO', '0%', 'Not Found!']
['KMISTERKA', ' KMISTERKA', '0%', 'Not Found!']
['LSZOTT', ' LSZOTT', '0%', 'Not Found!']
['JAYSILVER', ' JAYSILVER', '0%', 'Not Found!']
['TIMGRAY', ' TIMGRAY', '0%', 'Not Found!']
['RENEEHORVATICH', ' RENEEHORVATICH', '0%', 'Not Found!']
['SCOTTFLEMING', ' SCOTTFLEMING', '0%', 'Not Found!']
['GARYJEFFRIES', ' GARYJEFFRIES', '0%', 'Not Found!']
['KRISTENSPURLOCK', ' KRISTENSPURLOCK', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['JODYSHUMWAY', ' JODYSHUMWAY', '0%', 'Not Found!']
['ALCO.CTX', ' ALCO.CTX', '0%', 'Both not found!']
['PPGGARA

['TIM.MILLS', 'TIM MILLS TIM.MILLS', '100%', 'Exactly match both names']
['PHANTOM', ' PHANTOM', '0%', 'Not Found!']
['HANTOM.UK', ' HANTOM.UK', '0%', 'Both not found!']
['PETROLAB.SERVICE', ' PETROLAB.SERVICE', '0%', 'Both not found!']
['SALES.CSI', 'SALES SALES.CSI', '56%', 'Matched firstname only']
['REICHERTAI.REFRACTOMETERS', ' REICHERTAI.REFRACTOMETERS', '0%', 'Both not found!']
['SINGAPORESALES.CTS', ' SINGAPORESALES.CTS', '0%', 'Both not found!']
['USASALES.CTS', ' USASALES.CTS', '0%', 'Both not found!']
['JOEB', 'JOE JOEB', '75%', 'Leading match from firstname']
['JSMITH', 'SMITH JSMITH', '83%', 'Trailing match from firstname']
['KKELLY', 'KELLY KKELLY', '83%', 'Trailing match from firstname']
['ASEIDEL', 'SEIDEL ASEIDEL', '86%', 'Trailing match from lastname']
['BMYERS', 'MYERS BMYERS', '83%', 'Trailing match from lastname']
['JDOE', 'DOE JDOE', '75%', 'Trailing match from lastname']
['KGARY', 'GARY KGARY', '80%', 'Trailing match from firstname']
['QGUO', 'GUO QGUO', '75%', '

['KBAKKER', 'BAKKER KBAKKER', '86%', 'Trailing match from lastname']
['LMAXWELL', 'MAXWELL LMAXWELL', '88%', 'Trailing match from firstname']
['TRAININGQUOTES', ' TRAININGQUOTES', '0%', 'Not Found!']
['LEARNING', ' LEARNING', '0%', 'Not Found!']
['JDOE', 'DOE JDOE', '75%', 'Trailing match from lastname']
['SUPPORT', ' SUPPORT', '0%', 'Not Found!']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['MCROSS', 'CROSS MCROSS', '83%', 'Trailing match from lastname']
['EMAILSALES', ' EMAILSALES', '0%', 'Not Found!']
['EMELY.ARIENZA', 'EMELY ARIENZA EMELY.ARIENZA', '100%', 'Exactly match both names']
['SUPPORT', ' SUPPORT', '0%', 'Not Found!']
['ROB.WARNER', 'ROB ROB.WARNER', '30%', 'Matched firstname only']
['ROBERT.VENTRONE', 'ROBERT ROBERT.VENTRONE', '40%', 'Matched firstname only']
['AVAILITYLEARNING', ' AVAILITYLEARNING', '0%', 'Not Found!']
['INBORNESALES', ' INBORNESALES', '0%', 'Not Found!']
['AACENROLLMENT', ' AACENROLLMENT', '0%', 'Not Found!']
['ALLISON.HOSAK', 'ALLIS

['GBELENKO', ' GBELENKO', '0%', 'Not Found!']
['GVANHOOSE', ' GVANHOOSE', '0%', 'Not Found!']
['JREID', 'REID JREID', '80%', 'Trailing match from firstname']
['MSHARMA', 'SHARMA MSHARMA', '86%', 'Trailing match from lastname']
['EANDERSON', 'ANDERSON EANDERSON', '89%', 'Trailing match from firstname']
['CONTACTUS', ' CONTACTUS', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['KENDALLH', 'KENDALL KENDALLH', '88%', 'Leading match from firstname']
['JIMW', 'JIM JIMW', '75%', 'Leading match from firstname']
['LAST', 'LAS LAST', '75%', 'Leading match from lastname']
['MATTHEWH', 'MATTHEW MATTHEWH', '88%', 'Leading match from firstname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['JANE.DOE', 'JANE DOE JANE.DOE', '100%', 'Exactly match both names']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['MA

...

['CONTACT', 'CONTACT CONTACT', '100%', 'Exact match from lastname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['SURFACE', ' SURFACE', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['KAROLEEN', ' KAROLEEN', '0%', 'Not Found!']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['DSECOR', ' DSECOR', '0%', 'Not Found!']
['EUGENE', 'EUGEN EUGENE', '83%', 'Leading match from firstname']
['CAREERS', ' CAREERS', '0%', 'Not Found!']
['JKOVALA', ' JKOVALA', '0%', 'Not Found!']
['STAMBLYN', ' STAMBLYN', '0%', 'Not Found!']
['MDUNN', 'DUNN MDUNN', '80%', 'Trailing match from lastname']
['MGIBSON', 'GIBSON MGIBSON', '86%', 'Trailing match from lastname']
['MBARALE', ' MBARALE', '0%', 'Not Found!']
['JHOGUE', ' JHOGUE', '0%', 'Not Found!']
['LFINDLAY', 'FINDLAY LFINDLAY', '88%', 'Trailing match from lastname']
['

['ANDREC', 'ANDRE ANDREC', '83%', 'Leading match from firstname']
['ADAM.BLACK', 'ADAM BLACK ADAM.BLACK', '100%', 'Exactly match both names']
['ENQUIRIES', ' ENQUIRIES', '0%', 'Not Found!']
['INQUIRIES', ' INQUIRIES', '0%', 'Not Found!']
['ANDREW.AUSTIN', 'ANDREW ANDREW.AUSTIN', '46%', 'Matched firstname only']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['SUPPORT', ' SUPPORT', '0%', 'Not Found!']
['CUSTOMER.HELP', 'HELP CUSTOMER.HELP', '31%', 'Matched namename only']
['MEMBERSHIPSUPPORT', ' MEMBERSHIPSUPPORT', '0%', 'Not Found!']
['MARK', 'MAR MARK', '75%', 'Leading match from firstname']
['BILL', 'BILL BILL', '100%', 'Exact match from firstname']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['MIKE.COPELAND', 'MIKE COPELAND MIKE.COPELAND', '100%', 'Exactly match both names']
['CSHOUSEDUNITS', ' CSHOUSEDUNITS', '0%', 'Not Found!']
['PATRICK.ROACH', 'PATRICK ROACH PATRICK.ROACH', '100%', '

['JAPANSALES', ' JAPANSALES', '0%', 'Not Found!']
['EUROSALES', ' EUROSALES', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['DVANBORTEL', ' DVANBORTEL', '0%', 'Not Found!']
['JSTEIER', ' JSTEIER', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['GREGORY_MOROSE', 'GREGORY GREGORY_MOROSE', '50%', 'Matched firstname only']
['CHRISTOPHER_HANSEN', 'CHRISTOPHER HANSEN CHRISTOPHER_HANSEN', '100%', 'Exactly match both names']
['VETERANS', ' VETERANS', '0%', 'Not Found!']
['WEBCONTENT', ' WEBCONTENT', '0%', 'Not Found!']
['KRISTIN_BOURGAULT', 'KRISTIN KRISTIN_BOURGAULT', '41%', 'Matched firstname only']
['MARY_MONTESALVO', 'MARY MARY_MONTESALVO', '27%', 'Matched firstname only']
['PATRICK_CASEY1', 'PATRICK PATRICK_CASEY1', '50%', 'Matched firstname only']
['GAIL_SHEEHY', 'GAIL SHEEHY GAIL_SHEEHY', '100%', 'Exactly match both names']
['JANET_SCHRENK', 'JANET JANET_SCHRENK', '38%', 'Matched firstname only']
['DONNA_MULLIN', 'DONN

['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['DRLORILEE', ' DRLORILEE', '0%', 'Not Found!']
['WILLWARN', ' WILLWARN', '0%', 'Not Found!']
['CCNM', ' CCNM', '0%', 'Not Found!']
['CONTACT', 'CONTACT CONTACT', '100%', 'Exact match from lastname']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['MANUELA.DOLLINGER', 'MANUELA MANUELA.DOLLINGER', '41%', 'Matched firstname only']
['2933INVESTOR.RELATIONS', ' 2933INVESTOR.RELATIONS', '0%', 'Both not found!']
['INVESTOR.RELATIONS', ' INVESTOR.RELATIONS', '0%', 'Both not found!']
['CHRISTOF.BACHMAIR', 'CHRISTOF CHRISTOF.BACHMAIR', '47%', 'Matched firstname only']
['GREGORY.DULL', 'GREGORY GREGORY.DULL', '58%', 'Matched firstname only']
['SALES', 'ALES SALES', '80%', 'Trailing match from firstname']
['BILTRITESALES', ' BILTRITESALES', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['JOHN.SOBOL', 'JOHN SOBOL JOHN.SOBOL', '100%', 'Exactly match both names']
['CUSTOMERSERVICE', ' CUSTOMERS

['SHAUNL', 'SHAUN SHAUNL', '83%', 'Leading match from firstname']
['WALTONHR', ' WALTONHR', '0%', 'Not Found!']
['INFO', 'INFO INFO', '100%', 'Exact match from firstname']
['ZOTEK', ' ZOTEK', '0%', 'Not Found!']
['CUSTSERV', ' CUSTSERV', '0%', 'Not Found!']
['MSDS', ' MSDS', '0%', 'Not Found!']
I----------------- STATS---------------------------------------------------------I
Downloaded the file in 132.57 seconds
I----------------- STATS---------------------------------------------------------I
